# Import modules

In [1]:
!pip install folium
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests 
from pandas.io.json import json_normalize 
import folium

     |████████████████████████████████| 94 kB 5.7 MB/s  eta 0:00:01


# Import data

In [2]:
CLIENT_ID = 'YXP233FPW4KRBNYVABA2DIRIUIPLLFHVKINGBL1NGGAOPR33' 
CLIENT_SECRET = 'EPEPZJ05SMSKGGF1HB2BHKLR3DINEKUM1KVI2XREOCD1SQVX' 
VERSION = '20210118'

In [3]:
LIMIT = 500 
cities = ['Los Angeles', 'New York', 'London', 'Milan', 'Barcelona']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        '4d4b7105d754a06376d81259') 
    results[city] = requests.get(url).json()

In [4]:
df_venues={}
for city in cities:
    venues = pd.json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

Create a map to display the nightlife spots in each city

In [5]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.Marker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.6,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of nightlife spots {city} = ", results[city]['response']['totalResults'])

Total number of nightlife spots Los Angeles =  138
Total number of nightlife spots New York =  197
Total number of nightlife spots London =  119
Total number of nightlife spots Milan =  99
Total number of nightlife spots Barcelona =  111


# Methodology

Display the maps of each city with a marker for each bar

In [6]:
maps[cities[0]]

In [7]:
maps[cities[1]]

In [8]:
maps[cities[2]]

In [9]:
maps[cities[3]]

In [10]:
maps[cities[4]]

Create a dataframe of the bar for each city

In [11]:
df_Los_Angeles=pd.DataFrame.from_dict(df_venues[cities[0]])
df_Los_Angeles.head()

,Name,Address,Lat,Lng
0,Wally's,447 N. Canon Drive,34.071475,-118.401782
1,Silver Lake Wine,2395 Glendale Blvd,34.100003,-118.259315
2,K&L Wine Merchants,1400 Vine St,34.096524,-118.326393
3,Met Her At A Bar,759 S La Brea Ave,34.060557,-118.344744
4,MacLeod Ale Brewing Co.,14741 Calvert St,34.181907,-118.454576


In [12]:
df_New_York=pd.DataFrame.from_dict(df_venues[cities[1]])
df_New_York.head()

,Name,Address,Lat,Lng
0,NR,339 E 75th St,40.770082,-73.954978
1,The Bar Room at Temple Court,123 Nassau St,40.711448,-74.006802
2,Dear Irving,55 Irving Pl,40.736089,-73.987324
3,Interboro Spirits and Ales,942 Grand St,40.712771,-73.937030
4,Attaboy,134 Eldridge St,40.718981,-73.991540


In [13]:
df_London=pd.DataFrame.from_dict(df_venues[cities[2]])
df_London.head()

,Name,Address,Lat,Lng
0,Scarfes Bar,252 High Holborn,51.517813,-0.118184
1,The Foyer & Reading Room,49 Brook St,51.512577,-0.147663
2,The Chesham Arms,15 Mehetabel Rd,51.547727,-0.050131
3,The Punch Room,10 Berners St,51.516905,-0.136151
4,The Beer Shop London,40 Nunhead Green,51.465729,-0.057836


In [14]:
df_Milan=pd.DataFrame.from_dict(df_venues[cities[3]])
df_Milan.head()

,Name,Address,Lat,Lng
0,Bulgari Lounge Bar,Via Privata Fratelli Gabba,45.470014,9.188943
1,Ceresio 7 Pools & Restaurant,"Via Ceresio, 7",45.484025,9.179849
2,La Prosciutteria,"Corso Giuseppe Garibaldi, 53",45.474152,9.183449
3,B Café,Via San Maurilio 20,45.462640,9.183381
4,Bicerin,Via Panfilo Castaldi 24,45.476988,9.202432


In [15]:
df_Barcelona=pd.DataFrame.from_dict(df_venues[cities[4]])
df_Barcelona.head()

,Name,Address,Lat,Lng
0,Lo Pinyol,"C. Torrent de L'Olla, 7",41.398598,2.161148
1,El Quinto Moño,"Carrer Villaroel, 96",41.384935,2.157213
2,Hotel Arts Ritz Club Lounge,NaN,41.386596,2.196598
3,Chivuo's,"C. Pintor Fortuny, 15",41.382961,2.169948
4,Blavis,Saragossa 85,41.403303,2.147298


Cluster the bars of each city

In [16]:
from sklearn.cluster import KMeans

In [17]:
k=5
k_cluster=df_Los_Angeles.drop(['Address', 'Name'],1)
kmeans=KMeans(n_clusters=k,random_state=0).fit(k_cluster)
kmeans.labels_
df_Los_Angeles.insert(0,'Cluster Labels',kmeans.labels_)
df_Los_Angeles

,Cluster Labels,Name,Address,Lat,Lng
0,4,Wally's,447 N. Canon Drive,34.071475,-118.401782
1,0,Silver Lake Wine,2395 Glendale Blvd,34.100003,-118.259315
2,4,K&L Wine Merchants,1400 Vine St,34.096524,-118.326393
3,4,Met Her At A Bar,759 S La Brea Ave,34.060557,-118.344744
4,2,MacLeod Ale Brewing Co.,14741 Calvert St,34.181907,-118.454576
5,0,Wurstküche,800 E 3rd St,34.045520,-118.236131
6,1,Casa Del Mar Hotel,1910 Ocean Way,34.006435,-118.491278
7,4,Das Bunker,4067 W Pico Blvd,34.047810,-118.324116
8,3,Brouwerij West,110 E 22nd St,33.727344,-118.279935
9,0,Modern Times Beer: The Dankness Dojo,832 S Olive St,34.044133,-118.257600


In [18]:
k=5
k_cluster=df_New_York.drop(['Address', 'Name'],1).reset_index(drop=True)
kmeans=KMeans(n_clusters=k,random_state=0).fit(k_cluster)
kmeans.labels_
df_New_York.insert(0,'Cluster Labels',kmeans.labels_)
df_New_York

,Cluster Labels,Name,Address,Lat,Lng
0,1,NR,339 E 75th St,40.770082,-73.954978
1,0,The Bar Room at Temple Court,123 Nassau St,40.711448,-74.006802
2,2,Dear Irving,55 Irving Pl,40.736089,-73.987324
3,4,Interboro Spirits and Ales,942 Grand St,40.712771,-73.937030
4,2,Attaboy,134 Eldridge St,40.718981,-73.991540
5,4,Fifth Hammer Brewing Company,10-28 46th Ave,40.746553,-73.951438
6,1,Bondurants,303 E 85th St,40.777119,-73.952043
7,2,Angel’s Share,8 Stuyvesant St,40.729755,-73.989360
8,0,The Raines Law Room,48 W 17th St,40.738745,-73.994483
9,3,Delta Sky360° Club,4 Penn Plz,40.750564,-73.992824


In [19]:
k=5
k_cluster=df_London.drop(['Address', 'Name'],1)
kmeans=KMeans(n_clusters=k,random_state=0).fit(k_cluster)
kmeans.labels_
df_London.insert(0,'Cluster Labels',kmeans.labels_)
df_London

,Cluster Labels,Name,Address,Lat,Lng
0,1,Scarfes Bar,252 High Holborn,51.517813,-0.118184
1,1,The Foyer & Reading Room,49 Brook St,51.512577,-0.147663
2,3,The Chesham Arms,15 Mehetabel Rd,51.547727,-0.050131
3,1,The Punch Room,10 Berners St,51.516905,-0.136151
4,0,The Beer Shop London,40 Nunhead Green,51.465729,-0.057836
5,1,Cahoots,13 Kingly Ct,51.512523,-0.138623
6,3,Howling Hops Tank Bar,Queens Yard,51.542863,-0.022620
7,1,The Southampton Arms,139 Highgate Rd,51.556167,-0.146089
8,0,The Mayflower,117 Rotherhithe St,51.501778,-0.053616
9,1,Gordon's Wine Bar,47 Villiers St,51.507911,-0.123293


In [20]:
k=5
k_cluster=df_Milan.drop(['Address', 'Name'],1)
kmeans=KMeans(n_clusters=k,random_state=0).fit(k_cluster)
kmeans.labels_
df_Milan.insert(0,'Cluster Labels',kmeans.labels_)
df_Milan

,Cluster Labels,Name,Address,Lat,Lng
0,3,Bulgari Lounge Bar,Via Privata Fratelli Gabba,45.470014,9.188943
1,3,Ceresio 7 Pools & Restaurant,"Via Ceresio, 7",45.484025,9.179849
2,3,La Prosciutteria,"Corso Giuseppe Garibaldi, 53",45.474152,9.183449
3,0,B Café,Via San Maurilio 20,45.462640,9.183381
4,3,Bicerin,Via Panfilo Castaldi 24,45.476988,9.202432
5,0,Iter,Via Fusetti 1,45.451133,9.174325
6,3,Blenderino Burger Bar,NaN,45.480764,9.187217
7,3,BeerShow,Via Borsieri 30,45.489050,9.189068
8,1,Caffe Margherita,Viale Toscana 17,45.446340,9.195253
9,1,El Büscia,Viale Beatrice D'Este 48,45.451835,9.196518


In [21]:
k=5
k_cluster=df_Barcelona.drop(['Address', 'Name'],1)
kmeans=KMeans(n_clusters=k,random_state=0).fit(k_cluster)
kmeans.labels_
df_Barcelona.insert(0,'Cluster Labels',kmeans.labels_)
df_Barcelona

,Cluster Labels,Name,Address,Lat,Lng
0,2,Lo Pinyol,"C. Torrent de L'Olla, 7",41.398598,2.161148
1,0,El Quinto Moño,"Carrer Villaroel, 96",41.384935,2.157213
2,3,Hotel Arts Ritz Club Lounge,NaN,41.386596,2.196598
3,0,Chivuo's,"C. Pintor Fortuny, 15",41.382961,2.169948
4,2,Blavis,Saragossa 85,41.403303,2.147298
5,3,Cañete,"C. Unió, 17",41.379154,2.173092
6,0,Lambicus Bar,"C. Tamarit, 107",41.375979,2.157355
7,4,La Mundana,"Vallespir, 93",41.381136,2.135893
8,1,Flor de Llúpol,"C. Mateu Ferran, 4",41.435005,2.188919
9,0,Bodega Armando,"C. Bisbe Laguarda, 3",41.379974,2.164891


Import libraries

In [22]:
import matplotlib.cm as cm
import matplotlib.colors as colors

Display the maps of each city with the night spots k-clustered

In [23]:
map_colored0=maps[cities[0]]

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, neighbourhood, cluster in zip(df_Los_Angeles['Lat'], df_Los_Angeles['Lng'], df_Los_Angeles['Address'], df_Los_Angeles['Cluster Labels']):
    label=folium.Popup('Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_colored0)
       
map_colored0

In [24]:
map_colored1=maps[cities[1]]

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, neighbourhood, cluster in zip(df_New_York['Lat'], df_New_York['Lng'], df_New_York['Address'], df_New_York['Cluster Labels']):
    label=folium.Popup('Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_colored1)
       
map_colored1

In [25]:
map_colored2=maps[cities[2]]

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, neighbourhood, cluster in zip(df_London['Lat'], df_London['Lng'], df_London['Address'], df_London['Cluster Labels']):
    label=folium.Popup('Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_colored2)
       
map_colored2

In [26]:
map_colored3=maps[cities[3]]

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, neighbourhood, cluster in zip(df_Milan['Lat'], df_Milan['Lng'], df_Milan['Address'], df_Milan['Cluster Labels']):
    label=folium.Popup('Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_colored3)
       
map_colored3

In [27]:
map_colored4=maps[cities[4]]

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors=[]
for lat, lon, neighbourhood, cluster in zip(df_Barcelona['Lat'], df_Barcelona['Lng'], df_Barcelona['Address'], df_Barcelona['Cluster Labels']):
    label=folium.Popup('Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_colored4)
       
map_colored4

Determine how many bar there are in each Los Angeles cluster

In [45]:
counts_LA = df_Los_Angeles['Cluster Labels'].value_counts().to_dict()
df_LA=pd.DataFrame.from_dict(counts_LA, orient='index')
df_LA.columns = ['Bar Number']
df_LA

,Bar Number
0,45
4,34
1,15
2,5
3,1


Determine how many bar there are in each New York cluster

In [46]:
counts_NY = df_New_York['Cluster Labels'].value_counts().to_dict()
df_NY=pd.DataFrame.from_dict(counts_NY, orient='index')
df_NY.columns = ['Bar Number']
df_NY

,Bar Number
2,32
0,29
3,22
1,12
4,5


Determine how many bar there are in each London cluster

In [47]:
counts_Lon = df_London['Cluster Labels'].value_counts().to_dict()
df_Lon=pd.DataFrame.from_dict(counts_Lon, orient='index')
df_Lon.columns = ['Bar Number']
df_Lon

,Bar Number
1,45
3,26
0,17
4,8
2,4


Determine how many bar there are in each Milan cluster

In [48]:
counts_Mil = df_Milan['Cluster Labels'].value_counts().to_dict()
df_Mil=pd.DataFrame.from_dict(counts_Mil, orient='index')
df_Mil.columns = ['Bar Number']
df_Mil

,Bar Number
1,31
3,30
0,22
2,14
4,2


Determine how many bar there are in each Barcelona cluster

In [49]:
counts_Bar = df_Barcelona['Cluster Labels'].value_counts().to_dict()
df_Bar=pd.DataFrame.from_dict(counts_LA, orient='index')
df_Bar.columns = ['Bar Number']
df_Bar

,Bar Number
0,45
4,34
1,15
2,5
3,1
